# Ushay Reconstrucción

# Filtrar por sl_contract_id

In [ ]:
import pandas as pd

# Ruta del archivo CSV
csv_path = "/workspaces/ushay_handling-2/data/dataset/2022-created_on_2024-06-04_02-05/cleaned_dataset.csv"

# Cargar el DataFrame desde el CSV
df = pd.read_csv(csv_path)

# Definir el `sl_contract_id` que quieres filtrar
sl_contract_id_filtrar = 1758481  # Cambia este ID según lo que necesites

# Filtrar el DataFrame
df_filtrado = df[df['sl_contract_id'] == sl_contract_id_filtrar]

# Verificar si hay datos
if df_filtrado.empty:
    print(f"❌ No se encontraron registros para `sl_contract_id` {sl_contract_id_filtrar}.")
else:
    print(f"✅ Se encontraron {len(df_filtrado)} registros para `sl_contract_id` {sl_contract_id_filtrar}.")
    
    # Mostrar las primeras filas
    print(df_filtrado.head())

# Guardar el resultado en un nuevo archivo CSV (opcional)
df_filtrado.to_csv(f"filtrado_{sl_contract_id_filtrar}.csv", index=False)
print(f"📂 Archivo guardado: filtrado_{sl_contract_id_filtrar}.csv")


In [ ]:
import hashlib
import os
import pandas as pd

def calcular_hash_sha1(contenido):
    """ Calcula el hash SHA-1 del contenido proporcionado. """
    sha1 = hashlib.sha1()
    sha1.update(contenido)
    return sha1.hexdigest()

def verificar_y_guardar_archivo(unico_archivo, destino_zip):
    """ Guarda un solo archivo .ushay como ZIP sin necesidad de unir partes. """
    with open(unico_archivo, 'rb') as f:
        contenido = f.read()

    # Buscar el separador "sercop"
    separador = b'sercop'
    posicion_separador = contenido.find(separador)

    if posicion_separador == -1:
        print(f"⚠️ Advertencia: No se encontró el separador 'sercop' en {unico_archivo}.")
        return None

    # Extraer los datos reales después de "sercop"
    datos = contenido[posicion_separador + len(separador):]

    # Guardar directamente el contenido como ZIP
    with open(destino_zip, 'wb') as f:
        f.write(datos)

    print(f"✅ Archivo ZIP guardado en: {destino_zip}")
    return destino_zip

def verificar_y_unir_partes(partes, destino_zip):
    """ Une las partes de un archivo .ushay y lo guarda como un archivo ZIP binario. """
    contenido_completo = b''
    hashes_verificados = {}

    for idx, parte in enumerate(partes):
        with open(parte, 'rb') as f:
            contenido = f.read()
        
        # Separar el hash y los datos usando el separador 'sercop'
        separador = b'sercop'
        posicion_separador = contenido.find(separador)

        if posicion_separador == -1:
            print(f"⚠️ Advertencia: No se encontró el separador 'sercop' en {parte}.")
            continue

        # Extraer hashes y datos
        hashes = contenido[:posicion_separador].decode('utf-8', errors='ignore')
        datos = contenido[posicion_separador + len(separador):]

        # Extraer los hashes en formato clave-valor
        hashes_array = hashes.split(';')
        nombre_parte_actual = f"part{idx}.ushay"
        hash_esperado = None

        for hash_info in hashes_array:
            if not hash_info.strip():
                continue
            partes_hash = hash_info.split(':')
            if len(partes_hash) == 2:
                nombre_parte, hash_valor = partes_hash
                hashes_verificados[nombre_parte] = hash_valor
                if nombre_parte == nombre_parte_actual:
                    hash_esperado = hash_valor

        if hash_esperado is None:
            print(f"❌ Error: No se encontró el hash para {nombre_parte_actual} en {parte}")
            continue

        # Verificar el hash de los datos actuales
        hash_calculado = calcular_hash_sha1(datos)
        if hash_calculado != hash_esperado:
            print(f"❌ Error: Hash mismatch en {nombre_parte_actual}, esperado {hash_esperado}, calculado {hash_calculado}")
            raise Exception(f"Hash mismatch en parte: {nombre_parte_actual}")

        # Concatenar los datos
        contenido_completo += datos

    # Guardar el archivo ZIP resultante
    with open(destino_zip, 'wb') as f:
        f.write(contenido_completo)

    print(f"✅ Archivo ZIP reconstruido en: {destino_zip}")
    return destino_zip

def modificar_ruta(ruta):
    """ Reemplaza 'ushay_handling' por 'ushay_handling-2' en las rutas """
    return ruta.replace("/workspaces/ushay_handling/", "/workspaces/ushay_handling-2/")

# Ruta del dataset CSV
csv_path = "/workspaces/ushay_handling-2/data/dataset/2022-created_on_2024-06-04_02-05/cleaned_dataset.csv"

# Leer el dataset
df = pd.read_csv(csv_path)

# Filtrar solo los archivos con extensión .ushay
df = df[df['extension'] == '.ushay']

# Modificar las rutas en el DataFrame
df['file_path'] = df['file_path'].apply(modificar_ruta)

# Agrupar por sl_contract_id (cada proceso)
grupos_procesos = df.groupby('sl_contract_id')

# Definir manualmente el sl_contract_id a probar
ejemplo_contract_id = 1758631  # Reemplázalo con el ID que quieras probar

directorio_destino = "/workspaces/ushay_handling-2/notebooks/meli/"
os.makedirs(directorio_destino, exist_ok=True)

if ejemplo_contract_id in grupos_procesos.groups:
    grupo = grupos_procesos.get_group(ejemplo_contract_id)
    
    # Verificar si es un solo archivo o partes
    archivo_unico = grupo['filename'].nunique() == 1 and grupo['hash'].nunique() == 1
    
    # Ordenar los archivos
    grupo = grupo.sort_values(by='sl_descripcion_del_archivo')
    partes = grupo['file_path'].tolist()

    # Imprimir información de las rutas
    print(f"Contract ID: {ejemplo_contract_id}")
    print("Vector de rutas ordenadas:")
    for parte in partes:
        print(parte)

    # Definir el nombre del archivo ZIP
    archivo_zip = os.path.join(directorio_destino, f"{ejemplo_contract_id}.zip")

    if archivo_unico:
        # Caso 1: Solo hay un archivo (mismo nombre y hash)
        try:
            archivo_zip_completo = verificar_y_guardar_archivo(partes[0], archivo_zip)
            if archivo_zip_completo:
                print(f"✅ Archivo ZIP guardado en: {archivo_zip_completo}")
        except Exception as e:
            print(f"❌ Error en proceso {ejemplo_contract_id}: {e}")
    else:
        # Caso 2: Hay múltiples partes, unirlas
        try:
            archivo_zip_completo = verificar_y_unir_partes(partes, archivo_zip)
            print(f"✅ Archivo ZIP reconstruido en: {archivo_zip_completo}")
        except Exception as e:
            print(f"❌ Error en proceso {ejemplo_contract_id}: {e}") 
else:
    print(f"❌ Error: No se encontró el sl_contract_id {ejemplo_contract_id} en el dataset.")


In [13]:
import pandas as pd

# Ruta del dataset CSV (ajusta si es necesario)
csv_path = "/workspaces/ushay_handling/data/dataset/2022-created_on_2024-06-04_02-05/cleaned_dataset.csv"

# Cargar el dataset
df = pd.read_csv(csv_path)

# Cantidad total de registros en el dataset
print(f"📊 Total de registros en el dataset: {len(df)}")

# Filtrar solo los archivos con extensión .ushay
df_ushay = df[df['extension'] == '.ushay']

# Cantidad de archivos con extensión .ushay
print(f"📂 Archivos con extensión .ushay: {len(df_ushay)}")

# Contar la cantidad de procesos únicos (sl_contract_id)
num_procesos = df_ushay['sl_contract_id'].nunique()
print(f"🔹 Procesos únicos con archivos .ushay: {num_procesos}")

# Mostrar algunas filas para ver qué datos están apareciendo
print("\n🔍 Ejemplo de datos filtrados:")
print(df_ushay[['sl_contract_id', 'sl_descripcion_del_archivo', 'file_path']].head(10))


📊 Total de registros en el dataset: 14342
📂 Archivos con extensión .ushay: 4811
🔹 Procesos únicos con archivos .ushay: 3685

🔍 Ejemplo de datos filtrados:
    sl_contract_id        sl_descripcion_del_archivo  \
2          1724739                       part1.ushay   
3          1724739                       part2.ushay   
4          1724739                       part0.ushay   
6          1757431     contratacion_pli_subinv.ushay   
9          1755652     contratacion_pli_subinv.ushay   
16         1755018     contratacion_pli_subinv.ushay   
21         1747349  contratacion_pli_subinv(4).ushay   
22         1746667     contratacion_pli_subinv.ushay   
27         1746522     contratacion_pli_subinv.ushay   
29         1746358  contratacion_pli_subinv(1).ushay   

                                            file_path  
2   /workspaces/ushay_handling/data/dataset/2022-c...  
3   /workspaces/ushay_handling/data/dataset/2022-c...  
4   /workspaces/ushay_handling/data/dataset/2022-c...  
6   

# Código Mejorado para Procesar Todos los <sl_contract_id>

In [ ]:
import hashlib
import os
import pandas as pd

def calcular_hash_sha1(contenido):
    """ Calcula el hash SHA-1 del contenido proporcionado. """
    sha1 = hashlib.sha1()
    sha1.update(contenido)
    return sha1.hexdigest()

def verificar_y_guardar_archivo(unico_archivo, destino_zip):
    """ Guarda un solo archivo .ushay como ZIP sin necesidad de unir partes. """
    with open(unico_archivo, 'rb') as f:
        contenido = f.read()

    # Buscar el separador "sercop"
    separador = b'sercop'
    posicion_separador = contenido.find(separador)

    if posicion_separador == -1:
        print(f"⚠️ Advertencia: No se encontró el separador 'sercop' en {unico_archivo}.")
        return None

    # Extraer los datos reales después de "sercop"
    datos = contenido[posicion_separador + len(separador):]

    # Guardar directamente el contenido como ZIP
    with open(destino_zip, 'wb') as f:
        f.write(datos)

    print(f"✅ Archivo ZIP guardado en: {destino_zip}")
    return destino_zip

def verificar_y_unir_partes(partes, destino_zip):
    """ Une las partes de un archivo .ushay y lo guarda como un archivo ZIP binario. """
    contenido_completo = b''

    for idx, parte in enumerate(partes):
        with open(parte, 'rb') as f:
            contenido = f.read()
        
        # Separar el hash y los datos usando el separador 'sercop'
        separador = b'sercop'
        posicion_separador = contenido.find(separador)

        if posicion_separador == -1:
            print(f"⚠️ Advertencia: No se encontró el separador 'sercop' en {parte}.")
            continue

        # Extraer los datos reales después de "sercop"
        datos = contenido[posicion_separador + len(separador):]

        # Concatenar los datos
        contenido_completo += datos

    # Guardar el archivo ZIP resultante
    with open(destino_zip, 'wb') as f:
        f.write(contenido_completo)

    print(f"✅ Archivo ZIP reconstruido en: {destino_zip}")
    return destino_zip

def modificar_ruta(ruta):
    """ Reemplaza 'ushay_handling' por 'ushay_handling-2' en las rutas """
    return ruta.replace("/workspaces/ushay_handling/", "/workspaces/ushay_handling-2/")

# 📂 **Ruta del dataset CSV**
csv_path = "/workspaces/ushay_handling-2/data/dataset/2022-created_on_2024-06-04_02-05/cleaned_dataset.csv"

# 📌 **Leer el dataset**
df = pd.read_csv(csv_path)

# 🔹 **Filtrar solo los archivos con extensión .ushay**
df = df[df['extension'] == '.ushay']

# 🔄 **Modificar las rutas en el DataFrame**
df['file_path'] = df['file_path'].apply(modificar_ruta)

# 📌 **Obtener todos los `sl_contract_id` únicos**
sl_contract_ids = df['sl_contract_id'].unique()

# 📂 **Directorio donde se guardarán los ZIPs**
directorio_destino = "/workspaces/ushay_handling-2/notebooks/meli/"
os.makedirs(directorio_destino, exist_ok=True)

# 🔥 **Procesar todos los `sl_contract_id` únicos**
for contract_id in sl_contract_ids:
    grupo = df[df['sl_contract_id'] == contract_id]
    
    # Verificar si es un solo archivo o partes
    archivo_unico = grupo['filename'].nunique() == 1 and grupo['hash'].nunique() == 1

    # Ordenar los archivos
    grupo = grupo.sort_values(by='sl_descripcion_del_archivo')
    partes = grupo['file_path'].tolist()

    # 📌 **Definir el nombre del archivo ZIP**
    archivo_zip = os.path.join(directorio_destino, f"{contract_id}.zip")

    if archivo_unico:
        # ✅ **Caso 1: Solo hay un archivo (mismo nombre y hash)**
        try:
            archivo_zip_completo = verificar_y_guardar_archivo(partes[0], archivo_zip)
            if archivo_zip_completo:
                print(f"✅ Archivo ZIP guardado en: {archivo_zip_completo}")
        except Exception as e:
            print(f"❌ Error en contrato {contract_id}: {e}")
    else:
        # ✅ **Caso 2: Hay múltiples partes, unirlas**
        try:
            archivo_zip_completo = verificar_y_unir_partes(partes, archivo_zip)
            print(f"✅ Archivo ZIP reconstruido en: {archivo_zip_completo}")
        except Exception as e:
            print(f"❌ Error en contrato {contract_id}: {e}")

print("🎯 **Procesamiento completado para todos los sl_contract_id únicos!**")


# Descomprimir .zips

In [3]:
import os
import zipfile

def descomprimir_archivos_zip(directorio_zip, directorio_destino):
    """Descomprime todos los archivos .zip en un directorio de salida sin crear carpetas innecesarias."""

    # Obtener la lista de archivos .zip en el directorio
    archivos_zip = [f for f in os.listdir(directorio_zip) if f.endswith('.zip')]

    if not archivos_zip:
        print("❌ No se encontraron archivos ZIP en el directorio.")
        return

    for archivo in archivos_zip:
        ruta_zip = os.path.join(directorio_zip, archivo)
        nombre_carpeta = os.path.splitext(archivo)[0]  # Nombre del ZIP sin la extensión
        ruta_destino = os.path.join(directorio_destino, nombre_carpeta)

        try:
            with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
                # Verificar si el archivo ZIP es válido antes de extraerlo
                if zip_ref.testzip() is not None:
                    print(f"❌ Error: {ruta_zip} está corrupto y no puede ser descomprimido.")
                    continue  # Salta este archivo corrupto

                # Solo crear la carpeta si el archivo ZIP es válido
                os.makedirs(ruta_destino, exist_ok=True)
                zip_ref.extractall(ruta_destino)
                print(f"✅ Archivo descomprimido: {ruta_zip} en {ruta_destino}")

        except zipfile.BadZipFile:
            print(f"❌ Error: {ruta_zip} no es un archivo ZIP válido.")

# Directorios de trabajo
directorio_zip = "/Users/melisaguerrero/Desktop/kapak/meli/"
directorio_destino = "/Users/melisaguerrero/Desktop/kapak/unzip/"

descomprimir_archivos_zip(directorio_zip, directorio_destino)


✅ Archivo descomprimido: /Users/melisaguerrero/Desktop/kapak/meli/1692435.zip en /Users/melisaguerrero/Desktop/kapak/unzip/1692435
✅ Archivo descomprimido: /Users/melisaguerrero/Desktop/kapak/meli/1728817.zip en /Users/melisaguerrero/Desktop/kapak/unzip/1728817
✅ Archivo descomprimido: /Users/melisaguerrero/Desktop/kapak/meli/1756305.zip en /Users/melisaguerrero/Desktop/kapak/unzip/1756305
❌ Error: /Users/melisaguerrero/Desktop/kapak/meli/1709615.zip no es un archivo ZIP válido.
✅ Archivo descomprimido: /Users/melisaguerrero/Desktop/kapak/meli/1754506.zip en /Users/melisaguerrero/Desktop/kapak/unzip/1754506
❌ Error: /Users/melisaguerrero/Desktop/kapak/meli/1712553.zip no es un archivo ZIP válido.
❌ Error: /Users/melisaguerrero/Desktop/kapak/meli/1693059.zip no es un archivo ZIP válido.
✅ Archivo descomprimido: /Users/melisaguerrero/Desktop/kapak/meli/1752611.zip en /Users/melisaguerrero/Desktop/kapak/unzip/1752611
✅ Archivo descomprimido: /Users/melisaguerrero/Desktop/kapak/meli/174955

# Realizar embeddings

In [1]:
import sys
import scipy
import gensim

print("Python Version:", sys.version)
print("SciPy Version:", scipy.__version__)
print("Gensim Version:", gensim.__version__)


Python Version: 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 10:14:12) [Clang 14.0.6 ]
SciPy Version: 1.11.4
Gensim Version: 4.3.2


In [3]:
import os
import gensim
import pdfplumber
from docx import Document
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd

# Ruta donde están los archivos descomprimidos
directorio_documentos = "/Users/melisaguerrero/Desktop/kapak/unzip"

# Archivos y extensiones a excluir
archivos_excluir = {"proceso.xml", "zip.info"}
extensiones_excluir = {".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".zip", ".rar", ".tar", ".gz", ".mp4", ".mp3"}

def extraer_texto_pdf(ruta_pdf):
    """Extrae el texto de un archivo PDF."""
    texto = ""
    try:
        with pdfplumber.open(ruta_pdf) as pdf:
            for pagina in pdf.pages:
                texto += pagina.extract_text() + "\n"
    except Exception as e:
        print(f"⚠️ No se pudo extraer texto de {ruta_pdf}: {e}")
    return texto.strip()

def extraer_texto_docx(ruta_docx):
    """Extrae el texto de un archivo DOCX."""
    texto = ""
    try:
        doc = Document(ruta_docx)
        for parrafo in doc.paragraphs:
            texto += parrafo.text + "\n"
    except Exception as e:
        print(f"⚠️ No se pudo extraer texto de {ruta_docx}: {e}")
    return texto.strip()

# Leer archivos dentro de los subdirectorios
documentos = []
for carpeta in os.listdir(directorio_documentos):
    ruta_carpeta = os.path.join(directorio_documentos, carpeta)

    if not os.path.isdir(ruta_carpeta):
        continue  # Omitir archivos sueltos en el directorio principal

    for archivo in os.listdir(ruta_carpeta):
        ruta_archivo = os.path.join(ruta_carpeta, archivo)

        if archivo in archivos_excluir or any(archivo.lower().endswith(ext) for ext in extensiones_excluir):
            print(f"⚠️ Archivo omitido por ser binario: {ruta_archivo}")
            continue  

        contenido = ""
        try:
            if archivo.lower().endswith(".txt"):
                with open(ruta_archivo, "r", encoding="utf-8") as f:
                    contenido = f.read()
            elif archivo.lower().endswith(".pdf"):
                contenido = extraer_texto_pdf(ruta_archivo)
            elif archivo.lower().endswith(".docx"):
                contenido = extraer_texto_docx(ruta_archivo)
            else:
                print(f"⚠️ Tipo de archivo no soportado: {ruta_archivo}")
                continue  

            if contenido.strip():  # Solo agregar documentos con contenido
                documentos.append(TaggedDocument(words=contenido.split(), tags=[f"{carpeta}/{archivo}"]))

        except Exception as e:
            print(f"⚠️ Error al procesar {ruta_archivo}: {e}")

# Entrenar modelo Doc2Vec solo si hay documentos válidos
if documentos:
    modelo_doc2vec = Doc2Vec(vector_size=100, window=5, min_count=2, workers=4, epochs=10)
    modelo_doc2vec.build_vocab(documentos)
    modelo_doc2vec.train(documentos, total_examples=modelo_doc2vec.corpus_count, epochs=modelo_doc2vec.epochs)

    #  Guardar el modelo entrenado
    modelo_doc2vec.save("modelo_doc2vec")

    # Generar embeddings para cada documento
    embeddings = {doc.tags[0]: modelo_doc2vec.infer_vector(doc.words) for doc in documentos}

    #  Convertir embeddings en un DataFrame
    df_embeddings = pd.DataFrame.from_dict(embeddings, orient='index')

    #  Guardar embeddings en un CSV
    df_embeddings.to_csv("embeddings.csv")

    print("Embeddings generados y guardados en 'embeddings.csv'")
else:
    print("No se encontraron documentos de texto para procesar.")


⚠️ Archivo omitido por ser binario: /Users/melisaguerrero/Desktop/kapak/unzip/1697305/proceso.xml
⚠️ Tipo de archivo no soportado: /Users/melisaguerrero/Desktop/kapak/unzip/1697305/8-Anexo 01-Terminos de Referencia-signed-WORD.doc
⚠️ Archivo omitido por ser binario: /Users/melisaguerrero/Desktop/kapak/unzip/1697305/LOGO-MSP.jpg
⚠️ Archivo omitido por ser binario: /Users/melisaguerrero/Desktop/kapak/unzip/1697305/zip.info
⚠️ Archivo omitido por ser binario: /Users/melisaguerrero/Desktop/kapak/unzip/1679227/proceso.xml
⚠️ Archivo omitido por ser binario: /Users/melisaguerrero/Desktop/kapak/unzip/1679227/zip.info
⚠️ Archivo omitido por ser binario: /Users/melisaguerrero/Desktop/kapak/unzip/1718396/proceso.xml
⚠️ Archivo omitido por ser binario: /Users/melisaguerrero/Desktop/kapak/unzip/1718396/logo2021.jpg
⚠️ Archivo omitido por ser binario: /Users/melisaguerrero/Desktop/kapak/unzip/1718396/zip.info
⚠️ Archivo omitido por ser binario: /Users/melisaguerrero/Desktop/kapak/unzip/1726499/proc